In [1]:
from birdcall.data import *
from birdcall.metrics import *

import pandas as pd

In [2]:
classes = pd.read_pickle('data/classes.pkl')
train_ds = MelspecPoolDataset(pd.read_pickle('data/train_set.pkl'), classes, len_mult=30)
valid_ds = MelspecPoolDataset(pd.read_pickle('data/val_set.pkl'), classes, len_mult=5)

In [3]:
len(train_ds), len(valid_ds)

(7920, 1320)

In [4]:
import torch
import torchvision
from torch import nn

In [5]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=NUM_WORKERS)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=2*16, shuffle=False, num_workers=NUM_WORKERS)

In [6]:
for b in train_dl: break
b[0].shape, b[1]

(torch.Size([16, 10, 3, 80, 212]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64))

In [7]:
b[0].mean(), b[0].std()

(tensor(0.2136), tensor(1.6780))

In [8]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(*list(torchvision.models.resnet34(True).children())[:-2])
        self.classifier = nn.Sequential(*[
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=0.5), nn.BatchNorm1d(512),
            nn.Linear(512, len(classes))
        ])
    
    def forward(self, x):
        bs, im_num, ch, y_dim, x_dim = x.shape
        x = self.cnn(x.view(-1, ch, y_dim, x_dim))
        x = x.mean((2,3))
        x = self.classifier(x)
        x = x.view(bs, im_num, -1)
        x = x.mean(-2)
        return x

In [9]:
model = Model().cuda()

In [10]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), 1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 10)

In [11]:
from sklearn.metrics import accuracy_score, f1_score

In [12]:
import time

In [ ]:
for epoch in range(330):
    t0 = time.time()
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        model.train()
        inputs, labels = data[0].cuda(), data[1].cuda()
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item()

        if i % len(train_dl) == len(train_dl)-1:
            model.eval();
            preds = []
            targs = []

            with torch.no_grad():
                for data in valid_dl:
                    inputs, labels = data[0].cuda(), data[1].cuda()
                    outputs = model(inputs)
                    preds.append(outputs.cpu().detach())
                    targs.append(labels.cpu().detach())

                preds = torch.cat(preds)
                targs = torch.cat(targs)
            
            accuracy = accuracy_score(preds.sigmoid() > 0.5, targs)
            f1 = f1_score(preds.sigmoid() > 0.5, targs, average='micro')
            print(f'[{epoch + 1}, {time.time() - t0:.1f}] loss: {running_loss / (len(train_dl)-1):.3f}, acc: {accuracy:.3f}, f1: {f1:.3f}')
            running_loss = 0.0
            
        if (epoch % 20 == 0) and (epoch != 0): torch.save(model.state_dict(), f'models/{epoch}_{round(f1, 2)}.pth')

[1, 143.8] loss: 0.146, acc: 0.000, f1: 0.005
[2, 139.4] loss: 0.025, acc: 0.000, f1: 0.015
[3, 139.4] loss: 0.025, acc: 0.000, f1: 0.004
[4, 140.3] loss: 0.025, acc: 0.000, f1: 0.006
[5, 138.3] loss: 0.025, acc: 0.000, f1: 0.024
[6, 140.3] loss: 0.024, acc: 0.000, f1: 0.010
[7, 139.1] loss: 0.023, acc: 0.000, f1: 0.022
[8, 140.5] loss: 0.023, acc: 0.002, f1: 0.017
[9, 139.7] loss: 0.022, acc: 0.000, f1: 0.007
[10, 138.9] loss: 0.022, acc: 0.000, f1: 0.021
[11, 139.2] loss: 0.021, acc: 0.001, f1: 0.024
[12, 138.7] loss: 0.021, acc: 0.000, f1: 0.008
[13, 140.9] loss: 0.021, acc: 0.004, f1: 0.026
[14, 140.4] loss: 0.020, acc: 0.005, f1: 0.025
[15, 138.7] loss: 0.020, acc: 0.008, f1: 0.029
[16, 140.1] loss: 0.020, acc: 0.008, f1: 0.029
[17, 140.3] loss: 0.019, acc: 0.011, f1: 0.045
[19, 140.1] loss: 0.018, acc: 0.036, f1: 0.118
[20, 139.5] loss: 0.018, acc: 0.039, f1: 0.038
[21, 386.1] loss: 0.018, acc: 0.062, f1: 0.043
[22, 139.2] loss: 0.017, acc: 0.035, f1: 0.013
[23, 139.7] loss: 0.01

[176, 139.9] loss: 0.002, acc: 0.470, f1: 0.604
[177, 139.5] loss: 0.002, acc: 0.506, f1: 0.629
[178, 140.7] loss: 0.002, acc: 0.486, f1: 0.623
[179, 141.0] loss: 0.002, acc: 0.520, f1: 0.643
[180, 141.2] loss: 0.002, acc: 0.496, f1: 0.625
[181, 386.7] loss: 0.002, acc: 0.528, f1: 0.646
[182, 140.1] loss: 0.002, acc: 0.500, f1: 0.624
[183, 139.9] loss: 0.002, acc: 0.478, f1: 0.608
[184, 139.7] loss: 0.002, acc: 0.492, f1: 0.619
[185, 139.9] loss: 0.002, acc: 0.505, f1: 0.635
[186, 141.0] loss: 0.002, acc: 0.492, f1: 0.616
[187, 141.5] loss: 0.002, acc: 0.461, f1: 0.614
[188, 140.7] loss: 0.002, acc: 0.516, f1: 0.630
[189, 140.1] loss: 0.002, acc: 0.502, f1: 0.626
[190, 139.5] loss: 0.002, acc: 0.502, f1: 0.615
[191, 140.0] loss: 0.002, acc: 0.523, f1: 0.646
[192, 140.2] loss: 0.002, acc: 0.488, f1: 0.610
[193, 140.5] loss: 0.002, acc: 0.504, f1: 0.639
[194, 141.1] loss: 0.002, acc: 0.492, f1: 0.629
[195, 140.6] loss: 0.002, acc: 0.492, f1: 0.612
[196, 141.6] loss: 0.002, acc: 0.524, f1

In [ ]:
torch.save(model.state_dict(), f'models/{epoch}_{round(f1, 2)}.pth'